In [18]:
import pandas as pd
import numpy as np

In [19]:
file = "data/Town_Hall_Rich_List_2023.xlsx"
data_df = pd.read_excel(file).fillna(0)

In [20]:
data_df = data_df \
    .groupby(['Council'], sort=False, as_index=False) \
    .agg(
        count_people=("Council", "count"),
        sum_tc=("Total", "sum"),
        mean_tc=("Total", "mean"),
        sd_tc=("Total", np.std)
).fillna(0)

In [24]:
data_df.sort_values(by="mean_tc", ascending=False)

,Council,count_people,sum_tc,mean_tc,sd_tc
341,Wyre Forest,4,893252.0,223313.000000,72184.329925
124,Guildford,5,1095759.0,219151.800000,217358.040324
144,High Peak,1,213309.0,213309.000000,0.000000
14,Barnet,8,1697635.0,212204.375000,29611.595185
211,North Lanarkshire,8,1577275.0,197159.375000,141318.011074
...,...,...,...,...,...
273,South Somerset,9,960000.0,106666.666667,5055.937104
156,Isles of Scilly,1,104954.0,104954.000000,0.000000
102,Eden,1,102527.0,102527.000000,0.000000
323,West Devon,1,101600.0,101600.000000,0.000000


In [29]:
data_df[(data_df["Council"] == "Sheffield")]

,Council,count_people,sum_tc,mean_tc,sd_tc
256,Sheffield,7,1151317.0,164473.857143,46207.75139
